In [1]:
#nbi:hide_in
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from warnings import simplefilter
import base64

In [2]:
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
#plt.rcParams["figure.dpi"]=138 #This is for specifying the dpi and size of the graph
plt.rcParams["figure.figsize"] = (13,7.5)
plt.rcParams.update({'font.size': 20})
plt.rcParams['axes.grid'] = True
plt.rcParams['lines.markersize'] = 12
plt.rcParams['axes.titlepad'] = 20 

In [3]:
#nbi:hide_in
def IMAC_R(city,Range='90%'):
    if Range=='90%':
        data=pd.read_csv('Raw_Cities.csv')
        complete_date=pd.date_range(start='12-01-2021 00:00',end='12-31-2022 23:00',freq='H')
        data=data.assign(Time=complete_date[744:])
        dec_vals=data.iloc[-744:][city]
        d = {'Time':complete_date[:744], city: dec_vals}
        december_df=pd.DataFrame(data=d)
        Complete_df=pd.concat((december_df,data))
        Complete_df['month']=pd.to_datetime(Complete_df['Time']).dt.month
        Complete_df['day']=pd.to_datetime(Complete_df['Time']).dt.day
        Complete_df['year']=pd.to_datetime(Complete_df['Time']).dt.year
        average_daily=Complete_df.groupby(['year','month','day'])[city].mean()
        rolling_mean_30=np.round(average_daily.rolling(30).mean(),2)
        w,x,=np.array(np.round(average_daily.iloc[31:],2)),np.array(rolling_mean_30.iloc[31:])
        dd=pd.date_range(start='01-01-2022',end='12-31-2022',freq='D')
        dday=dd.day
        dmon=dd.month_name()
        T_neutral=np.round(x*0.42+17.6,2)
        T_plus=T_neutral+2.15
        T_minus=T_neutral-2.15
        numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
        Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP (°C)','TOP(°C) 90% Acceptability Upper Band','TOP(°C) 90% Acceptability Lower Band'])
        x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y
    elif Range=='80%':
        data=pd.read_csv('Raw_Cities.csv')
        complete_date=pd.date_range(start='12-01-2021 00:00',end='12-31-2022 23:00',freq='H')
        data=data.assign(Time=complete_date[744:])
        dec_vals=data.iloc[-744:][city]
        d = {'Time':complete_date[:744], city: dec_vals}
        december_df=pd.DataFrame(data=d)
        Complete_df=pd.concat((december_df,data))
        Complete_df['month']=pd.to_datetime(Complete_df['Time']).dt.month
        Complete_df['day']=pd.to_datetime(Complete_df['Time']).dt.day
        Complete_df['year']=pd.to_datetime(Complete_df['Time']).dt.year
        average_daily=Complete_df.groupby(['year','month','day'])[city].mean()
        rolling_mean_30=np.round(average_daily.rolling(30).mean(),2)
        w,x,=np.array(np.round(average_daily.iloc[31:],2)),np.array(rolling_mean_30.iloc[31:])
        dd=pd.date_range(start='01-01-2022',end='12-31-2022',freq='D')
        dday=dd.day
        dmon=dd.month_name()
        T_neutral=np.round(x*0.42+17.6,2)
        T_plus=T_neutral+3.60
        T_minus=T_neutral-3.60
        numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
        Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP (°C)','TOP(°C) 80% Acceptability Upper Band','TOP(°C) 80% Acceptability Lower Band'])
        x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
        y=x.drop(['Day','Rolling_mean'],axis=1)
        return y

In [4]:
def graph_IMAC_R(city,Range='90%'):
    if Range=='90%':
        x1=np.linspace(1,12,12)
        y1=IMAC_R(city,Range='90%')['Monthly Average Outdoor DBT (°C)']
        y2=IMAC_R(city,Range='90%')['Monthly Average Neutral Operative Temperature TOP (°C)']
        y3=IMAC_R(city,Range='90%')['TOP(°C) 90% Acceptability Upper Band']
        y4=IMAC_R(city,Range='90%')['TOP(°C) 90% Acceptability Lower Band']
        plt.plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
        plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C)',marker='o')
        plt.xlabel('Month',loc='center')
        plt.ylabel('Temperature $^{\circ}C$',loc='center')
        plt.fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='90% Acceptability Range $T_{op}$(°C)')
        plt.grid(alpha=0.3)
        labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        plt.xticks(x1, labels, rotation='vertical')
        plt.xlim(1,12)
        plt.title('India Model for Adaptive Comfort for Residential Buildings – MM Buildings in '+str(city),fontsize=20)
        plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
        plt.show()
    elif Range=='80%':
        x1=np.linspace(1,12,12)
        y1=IMAC_R(city,Range)['Monthly Average Outdoor DBT (°C)']
        y2=IMAC_R(city,Range)['Monthly Average Neutral Operative Temperature TOP (°C)']
        y3=IMAC_R(city,Range)['TOP(°C) 80% Acceptability Upper Band']
        y4=IMAC_R(city,Range)['TOP(°C) 80% Acceptability Lower Band']
        plt.plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
        plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C)',marker='o')
        plt.xlabel('Month',loc='center')
        plt.ylabel('Temperature $^{\circ}C$',loc='center')
        plt.fill_between(x1,y3,y4,color='firebrick',alpha=0.7,label='80% Acceptability Range $T_{op}$ (°C)')
        plt.fill_between(x1,IMAC_R(city,Range='90%')['TOP(°C) 90% Acceptability Upper Band'],IMAC_R(city,Range='90%')['TOP(°C) 90% Acceptability Lower Band'],color='wheat',alpha=0.7,label='90% Acceptability Range $T_{op}$(°C)')
        plt.grid(alpha=0.3)
        labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        plt.xticks(x1, labels, rotation='vertical')
        plt.xlim(1,12)
        plt.title('India Model for Adaptive Comfort for Residential Buildings – MM Buildings in '+str(city),fontsize=20)
        plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
        plt.show()

In [5]:
#nbi:hide_in
def NV(T):
    return 0.54*T+12.83
def MM(T):
    return 0.28*T+17.87
def AC(T):
    return 0.05*T+23.96

In [6]:
def IMAC(city,building='all',Range='90%'):
    data=pd.read_csv('Raw_Cities.csv')
    complete_date=pd.date_range(start='12-01-2021 00:00',end='12-31-2022 23:00',freq='H')
    data=data.assign(Time=complete_date[744:])
    dec_vals=data.iloc[-744:][city]
    d = {'Time':complete_date[:744], city: dec_vals}
    december_df=pd.DataFrame(data=d)
    Complete_df=pd.concat((december_df,data))
    Complete_df['month']=pd.to_datetime(Complete_df['Time']).dt.month
    Complete_df['day']=pd.to_datetime(Complete_df['Time']).dt.day
    Complete_df['year']=pd.to_datetime(Complete_df['Time']).dt.year
    average_daily=Complete_df.groupby(['year','month','day'])[city].mean()
    rolling_mean_30=np.round(average_daily.rolling(30).mean(),2)
    w,x,=np.array(np.round(average_daily.iloc[31:],2)),np.array(rolling_mean_30.iloc[31:])
    dd=pd.date_range(start='01-01-2022',end='12-31-2022',freq='D')
    dday=dd.day
    dmon=dd.month_name()
    if Range=='90%':
            if building=='NV':
                T_neutral=np.round(NV(x),1)
                T_plus=T_neutral+2.4
                T_minus=T_neutral-2.4
                numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 90% Acceptability Upper Band','TOP(°C) 90% Acceptability Lower Band'])
                x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                y=x.drop(['Day','Rolling_mean'],axis=1)
                return y
            elif building=='MM':
                T_neutral=np.round(MM(x),1)
                T_plus=T_neutral+3.5
                T_minus=T_neutral-3.5
                numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 90% Acceptability Upper Band','TOP(°C) 90% Acceptability Lower Band'])
                x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                y=x.drop(['Day','Rolling_mean'],axis=1)
                return y
            elif building=='AC':
                T_neutral=np.round(AC(x),1)
                T_plus=T_neutral+2.1
                T_minus=T_neutral-2.1
                numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 90% Acceptability Upper Band','TOP(°C) 90% Acceptability Lower Band'])
                x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                y=x.drop(['Day','Rolling_mean'],axis=1)
                return y
            elif building=='all':
                T_neutral_NV=np.round(NV(x),1)
                T_neutral_MM=np.round(MM(x),1)
                T_neutral_AC=np.round(AC(x),1)
                T_plus_NV=T_neutral_NV+2.4
                T_plus_MM=T_neutral_MM+3.5
                T_plus_AC=T_neutral_AC+2.1
                T_minus_NV=T_neutral_NV-2.4
                T_minus_MM=T_neutral_MM-3.5
                T_minus_AC=T_neutral_AC-2.1
                lst=(dmon,dday,w,x,T_neutral_NV,T_plus_NV,T_minus_NV,T_neutral_MM,T_plus_MM,T_minus_MM,T_neutral_AC,T_plus_AC,T_minus_AC)
                columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','NV Neutral','NV TOP(°C) 90% Acceptability Upper Band','NV TOP(°C) 90% Acceptability Lower Band','MM Neutral','MM TOP(°C) 90% Acceptability Upper Band','MM TOP(°C) 90% Acceptability Lower Band','AC Neutral','AC TOP(°C) 90% Acceptability Upper Band','AC TOP(°C) 90% Acceptability Lower Band']
                numpy_array=(np.column_stack(lst))
                Final_Data=pd.DataFrame(numpy_array,columns=columns)
                x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                y=x.drop(['Day','Rolling_mean'],axis=1)
                return y
    elif Range=='80%':
        if building=='NV':
                    T_neutral=np.round(NV(x),1)
                    T_plus=T_neutral+4.1
                    T_minus=T_neutral-4.1
                    numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                    Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 80% Acceptability Upper Band','TOP(°C) 80% Acceptability Lower Band'])
                    x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                    y=x.drop(['Day','Rolling_mean'],axis=1)
                    return y
        elif building=='MM':
                    T_neutral=np.round(MM(x),1)
                    T_plus=T_neutral+5.9
                    T_minus=T_neutral-5.9
                    numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                    Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 80% Acceptability Upper Band','TOP(°C) 80% Acceptability Lower Band'])
                    x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                    y=x.drop(['Day','Rolling_mean'],axis=1)
                    return y
        elif building=='AC':
                    T_neutral=np.round(AC(x),1)
                    T_plus=T_neutral+3.6
                    T_minus=T_neutral-3.6
                    numpy_array=(np.column_stack((dmon,dday,w,x,T_neutral,T_plus,T_minus)))
                    Final_Data=pd.DataFrame(numpy_array,columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','Monthly Average Neutral Operative Temperature TOP(°C)','TOP(°C) 80% Acceptability Upper Band','TOP(°C) 80% Acceptability Lower Band'])
                    x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                    y=x.drop(['Day','Rolling_mean'],axis=1)
                    return y
        elif building=='all':
                    T_neutral_NV=np.round(NV(x),1)
                    T_neutral_MM=np.round(MM(x),1)
                    T_neutral_AC=np.round(AC(x),1)
                    T_plus_NV=T_neutral_NV+4.1
                    T_plus_MM=T_neutral_MM+5.9
                    T_plus_AC=T_neutral_AC+3.6
                    T_minus_NV=T_neutral_NV-4.1
                    T_minus_MM=T_neutral_MM-5.9
                    T_minus_AC=T_neutral_AC-3.6
                    lst=(dmon,dday,w,x,T_neutral_NV,T_plus_NV,T_minus_NV,T_neutral_MM,T_plus_MM,T_minus_MM,T_neutral_AC,T_plus_AC,T_minus_AC)
                    columns=['Month','Day','Monthly Average Outdoor DBT (°C)','Rolling_mean','NV Neutral','NV TOP(°C) 80% Acceptability Upper Band','NV TOP(°C) 80% Acceptability Lower Band','MM Neutral','MM TOP(°C) 80% Acceptability Upper Band','MM TOP(°C) 80% Acceptability Lower Band','AC Neutral','AC TOP(°C) 80% Acceptability Upper Band','AC TOP(°C) 80% Acceptability Lower Band']
                    numpy_array=(np.column_stack(lst))
                    Final_Data=pd.DataFrame(numpy_array,columns=columns)
                    x=np.round(Final_Data.groupby('Month',sort=False).mean(),1)
                    y=x.drop(['Day','Rolling_mean'],axis=1)
                    return y

In [7]:
def graph_IMAC(city,building,Range):
    if Range=='90%':
        if building!='all':
            x1=np.linspace(1,12,12)
            y1=IMAC(city,building,Range)['Monthly Average Outdoor DBT (°C)']
            y2=IMAC(city,building,Range)['Monthly Average Neutral Operative Temperature TOP(°C)']
            y3=IMAC(city,building,Range)['TOP(°C) 90% Acceptability Upper Band']
            y4=IMAC(city,building,Range)['TOP(°C) 90% Acceptability Lower Band']
            plt.plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            plt.xlabel('Month',loc='center')
            plt.ylabel('Temperature $^{\circ}C$',loc='center')
            plt.fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='$T_{op}$(°C) 90% Acceptability Range ')
            plt.grid(alpha=0.3)
            labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
            plt.xticks(x1, labels, rotation='vertical')
            plt.xlim(1,12)
            plt.title('India Model for Adaptive Comfort for Commercial Buildings – '+str(building)+' in '+str(city),fontsize=20)
            plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
            plt.show()
        else:
            fig,ax=plt.subplots(3,sharex=True,sharey=True)
            x1=np.linspace(1,12,12)
            y1=IMAC(city,'all',Range)['Monthly Average Outdoor DBT (°C)']
            y2=IMAC(city,'all',Range)['NV Neutral']
            y3=IMAC(city,'all',Range)['NV TOP(°C) 90% Acceptability Upper Band']
            y4=IMAC(city,'all',Range)['NV TOP(°C) 90% Acceptability Lower Band']
            y5=IMAC(city,'all',Range)['MM Neutral']
            y6=IMAC(city,'all',Range)['MM TOP(°C) 90% Acceptability Upper Band']
            y7=IMAC(city,'all',Range)['NV TOP(°C) 90% Acceptability Lower Band']
            y8=IMAC(city,'all',Range)['AC Neutral']
            y9=IMAC(city,'all',Range)['AC TOP(°C) 90% Acceptability Upper Band']
            y10=IMAC(city,'all',Range)['AC TOP(°C) 90% Acceptability Lower Band']
            ax[0].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[0].plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[0].fill_between(x1,y3,y4,color='wheat',alpha=0.7,label='$T_{op}$(°C) 90% Acceptability Range ')

            ax[1].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[1].plot(x1,y5,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[1].fill_between(x1,y6,y7,color='wheat',alpha=0.7,label='$T_{op}$(°C) 90% Acceptability Range ')

            ax[2].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[2].plot(x1,y8,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[2].fill_between(x1,y9,y10,color='wheat',alpha=0.7,label='$T_{op}$(°C) 90% Acceptability Range ')

            ax[0].set_title('Naturally Ventillated')
            ax[1].set_title('Mix Mode')
            ax[2].set_title('Air Conditioned')

            labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
            plt.setp(ax,xticks=[1,2,3,4,5,6,7,8,9,10,11,12],xticklabels=labels,yticks=np.round((np.linspace(max(y1)+0.5,min(y1)-0.5,4)),0),xlim=(1,12))
            fig.supxlabel('Month')
            fig.supylabel('Temperature $^{\circ}C$')
            fig.tight_layout()
            plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
            plt.show()  
    if Range=='80%':
        if building!='all':
            x1=np.linspace(1,12,12)
            y1=IMAC(city,building,Range)['Monthly Average Outdoor DBT (°C)']
            y2=IMAC(city,building,Range)['Monthly Average Neutral Operative Temperature TOP(°C)']
            y3=IMAC(city,building,Range)['TOP(°C) 80% Acceptability Upper Band']
            y4=IMAC(city,building,Range)['TOP(°C) 80% Acceptability Lower Band']
            plt.plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            plt.plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            plt.xlabel('Month',loc='center')
            plt.ylabel('Temperature $^{\circ}C$',loc='center')
            plt.fill_between(x1,y3,y4,color='firebrick',alpha=0.7,label='80% Acceptability Range $T_{op}$(°C)')
            plt.fill_between(x1,IMAC(city,building,Range='90%')['TOP(°C) 90% Acceptability Upper Band'],IMAC(city,building,Range='90%')['TOP(°C) 90% Acceptability Lower Band'],color='wheat',alpha=0.7,label='90% Acceptability Range $T_{op}$(°C)')
            plt.grid(alpha=0.3)
            labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
            plt.xticks(x1, labels, rotation='vertical')
            plt.xlim(1,12)
            plt.title('India Model for Adaptive Comfort for Commercial Buildings – '+str(building)+' in '+str(city),fontsize=20)
            plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
            plt.show()
        else:
            fig,ax=plt.subplots(3,sharex=True,sharey=True)
            x1=np.linspace(1,12,12)
            y1=IMAC(city,'all',Range)['Monthly Average Outdoor DBT (°C)']
            y2=IMAC(city,'all',Range)['NV Neutral']
            y3=IMAC(city,'all',Range)['NV TOP(°C) 80% Acceptability Upper Band']
            y4=IMAC(city,'all',Range)['NV TOP(°C) 80% Acceptability Lower Band']
            y5=IMAC(city,'all',Range)['MM Neutral']
            y6=IMAC(city,'all',Range)['MM TOP(°C) 80% Acceptability Upper Band']
            y7=IMAC(city,'all',Range)['NV TOP(°C) 80% Acceptability Lower Band']
            y8=IMAC(city,'all',Range)['AC Neutral']
            y9=IMAC(city,'all',Range)['AC TOP(°C) 80% Acceptability Upper Band']
            y10=IMAC(city,'all',Range)['AC TOP(°C) 80% Acceptability Lower Band']
            ax[0].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[0].plot(x1,y2,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[0].fill_between(x1,y3,y4,color='firebrick',alpha=0.7,label='$T_{op}$(°C) 80% Acceptability Range ')

            ax[1].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[1].plot(x1,y5,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[1].fill_between(x1,y6,y7,color='firebrick',alpha=0.7,label='$T_{op}$(°C) 80% Acceptability Range ')

            ax[2].plot(x1,y1,ls='--',c='k',label='Monthly Average Outdoor DBT (°C)',marker='o')
            ax[2].plot(x1,y8,ls='-',c='goldenrod',label='Monthly Average Neutral Operative Temperature $T_{op}$ (°C) ',marker='o')
            ax[2].fill_between(x1,y9,y10,color='firebrick',alpha=0.7,label='$T_{op}$(°C) 80% Acceptability Range ')

            ax[0].set_title('Naturally Ventillated')
            ax[1].set_title('Mix Mode')
            ax[2].set_title('Air Conditioned')

            labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
            plt.setp(ax,xticks=[1,2,3,4,5,6,7,8,9,10,11,12],xticklabels=labels,yticks=np.round((np.linspace(max(y1)+0.5,min(y1)-0.5,4)),0),xlim=(1,12))
            fig.supxlabel('Month')
            fig.supylabel('Temperature $^{\circ}C$')
            fig.tight_layout()
            plt.legend(loc = 'lower center', bbox_to_anchor = (0,-0.31,1,1),bbox_transform = plt.gcf().transFigure )
            plt.show()   

In [8]:
data=pd.read_csv('Raw_Cities.csv')
del data['Hours']
x=data.columns
b_type=widgets.Dropdown(
    options=[('Naturally Ventilated','NV'), ('Mixed Mode','MM'), ('Air Conditioned','AC'),('All','all')],
    value='NV',
    description='Building Operation Mode:',style={'description_width': 'initial'})
city_list=widgets.Dropdown(
    #value='Ahmedabad',
    placeholder='Type your city',
    options=x,
    description='City:',
    ensure_option=True,
    disabled=False
)


comfort_range=widgets.Dropdown(
    options=[('80% feel comfortable','80%'), ('90% feel comfortable','90%')],
    value='80%',
    description='Range:',)

get_data_button_1=widgets.Button(description='Get Results')
get_data_button_2=widgets.Button(description='Get Results')
download_button_1=widgets.Output()
download_button_2=widgets.Output()

In [9]:
#nbi:hide_in
out = widgets.Output()
out1=widgets.Output()
out1.layout.width = 'calc(95% + 14ex)'

In [10]:
def create_download_link_2(city,Range):  
    x=IMAC_R(city,Range).to_csv(city+' '+Range)
    filename=city+' '+Range 
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}<button class="lm-Widget p-Widget jupyter-widgets jupyter-button widget-button">Download Data</button></a>'
    html = html.format(payload=payload,title='',filename=filename)
    return HTML(html)

def create_download_link_1(city,building,Range):  
    x=IMAC(city,building,Range).to_csv(city+' '+Range+' '+building )
    filename=city+' '+Range+' '+building 
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}<button class="lm-Widget p-Widget jupyter-widgets jupyter-button widget-button">Download Data</button></a>'
    html = html.format(payload=payload,title='',filename=filename)
    return HTML(html)

In [11]:
#nbi:hide_in
def on_button_clicked_1(b):
    with out:
        clear_output()
        display(IMAC(city_list.value,building=b_type.value,Range=comfort_range.value))
    with out1: 
        clear_output()
        (graph_IMAC(city_list.value,building=b_type.value,Range=comfort_range.value))
        
    with download_button_1:
        clear_output()
        display(create_download_link_1(city_list.value,building=b_type.value,Range=comfort_range.value))

get_data_button_1.on_click(on_button_clicked_1)

def on_button_clicked_2(b):
    with out:
        clear_output()
        display(IMAC_R(city_list.value,Range=comfort_range.value))
    with out1:
        clear_output()
        (graph_IMAC_R(city_list.value,Range=comfort_range.value))
        
    with download_button_2:
        clear_output()
        display(create_download_link_2(city_list.value,Range=comfort_range.value))
        
get_data_button_2.on_click(on_button_clicked_2)

In [12]:
#nbi:hide_in
inbox1=widgets.HBox([get_data_button_1,download_button_1])
inbox2=widgets.HBox([get_data_button_2,download_button_2])
box1 = widgets.VBox([city_list,b_type,comfort_range,inbox1,out])
box2=widgets.VBox([city_list,comfort_range,inbox2,out])
about=widgets.HBox()
references=widgets.HBox()
Rawdata=widgets.HBox()

In [13]:
tab_nest = widgets.Tab()
tab_nest.children = [box1, box2,]
tab_nest.set_title(0, 'Commercial Building')
tab_nest.set_title(1, 'Residential Building')

<center><img src="FOOTER.png" alt="logo" style="width:1000px;height:95px;"><center>

In [14]:
widgets.AppLayout(header=None,left_sidebar=(tab_nest),center=out1,right_sidebar=None,footer=None)

AppLayout(children=(Tab(children=(VBox(children=(Dropdown(description='City:', options=('Agartala', 'Agatti Is…

<style>
body, h1, h2, h3, h4, h5, h6  {
  font-family: "Times New Roman", Times, serif;
}
</style>
<center><h3>About the Tool:</h3> <center>
    
<p align="justify">
    
Buildings consume 40% of the total energy generated globally. A significant proportion of this consumption is utilized in order to maintain comfortable indoor temperatures via cooling or heating. A building can be classified as being in one of the three ‘Building Operation Modes’ depending on the presence of Air Conditioning. The ‘Natural Ventilation’ mode implies the absence of Air-Conditioning as well as assisted mechanical ventilation; conversely the ‘Air Conditioned’  mode implies that comfort is achieved wholly by the use of artificial mechanisms; the ‘Mixed Mode’ of operations is a combination of the two aforementioned modes, suggesting that Air Conditioning is used over specific spaces or during certain time-periods.
    
</p> 
    
<p align="justify">
    
Studies have shown that indoor comfort temperatures correlate strongly to outdoor weather conditions. The aim of this tool is to calculate indoor comfort temperatures or ‘Average Neutral Operative Temperatures’ for 160 Indian cities and towns and provide temperature ranges where 90% or 80% of occupants would feel comfortable in a building. This tool calculates these temperatures for commercial as well as residential buildings using a region’s Typical meteorological year (TMY) data. While the commercial building type includes three modes of operations (Naturally Ventilated, Air Conditioned, and Mixed Mode), the residential building type assumes a ‘Mix Mode’ of indoor operations. 
    
</p> 

<p align="justify">
    
The tool provides a table with the calculated temperatures and ranges and provides the visualisation for the same. A user may download this calculated data in the form of a .csv file by using the ‘Download Data’ feature. 
    
</p>                             

 <style>
body, h1, h2, h3, h4, h5, h6  {
  font-family: "Times New Roman", Times, serif;
}
</style>
 
 <center><h3>References:</h3> <center>
    <ol> 
  <p><cite align="justify"> 1.  Manu, S., Shukla, Y., Rawal, R., Thomas, L., & de Dear, R. (2016). Field studies of thermal comfort across multiple climate zones for the subcontinent: India Model for Adaptive Comfort (IMAC). Building And Environment, 98, 55-70.<a href="https://doi.org/10.1016/j.buildenv.2015.12.019"> Access Paper </a></cite>
</p> 
     
  <p><cite align="justify">2. Rawal, R., Shukla, Y., Vardhan, V., Asrani, S., Schweiker, M., & de Dear, R. et al. (2022). Adaptive thermal comfort model based on field studies in five climate zones across India. Building And Environment, 219, 109187. <a href="https://doi.org/10.1016/j.buildenv.2022.109187">Access Paper</a></cite></p>

<style>
body, h1, h2, h3, h4, h5, h6  {
  font-family: "Times New Roman", Times, serif;
}
</style>
<center><h3>Link to Raw-Data:</h3> <center>
<p align="justify">
The Typical Meteorological Year (TMY) data was extracted from <a href="https://climate.onebuilding.org/WMO_Region_2_Asia/IND_India/index.html">HERE</a>. Choose the file name with the following naming convention: <strong>"IND_[State Name]_[Place Name].[some number]_TMYx.2007-2021.zip"</strong>. Unzipping this file will lead to an .epw file whose data can be extracted in any .csv reader. The drybulb temperature data collected on an hourly basis was used for this project. If the 2007-2021 file is unavailable for a given region then choose the file that has the most recent year tag.</p>

<left><span style="font-size:10px">© 2022&nbsp;Tool developed by Spandan Pandya under the guidance of Prof. Rajan Rawal and Dr. Yash Shukla.</span><left>